> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [4]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv("salaries.csv")

# Exibir as primeiras linhas
print(df.head())

# Exibir informações sobre o DataFrame
print(df.info())


  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 13 columns)

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    String,
    Float,
    Date,
    Enum,
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import enum
import pandas as pd
from datetime import datetime

# Definir a base do ORM
Base = declarative_base()

# Definir as Enums com os valores únicos
class SexEnum(enum.Enum):
    F = "F"
    M = "M"

class DesignationEnum(enum.Enum):
    ANALYST = "Analyst"
    SENIOR_ANALYST = "Senior Analyst"
    MANAGER = "Manager"

class UnitEnum(enum.Enum):
    IT = "IT"
    FINANCE = "Finance"
    MARKETING = "Marketing"

# Definir o modelo para os dados
class Employee(Base):
    __tablename__ = 'employees'

    # Campos do modelo
    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    sex = Column(Enum(SexEnum), nullable=False)
    doj = Column(Date, nullable=False)  # Data de ingresso
    current_date = Column(Date, nullable=False)  # Data atual
    designation = Column(Enum(DesignationEnum), nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnitEnum), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)

    def __repr__(self):
        return f"<Employee(name={self.first_name} {self.last_name}, designation={self.designation})>"

# Criar o banco de dados (SQLite como exemplo)
engine = create_engine('sqlite:///employees.db')

# Criar as tabelas no banco de dados
Base.metadata.create_all(engine)

# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()

# Carregar o CSV
df = pd.read_csv("salaries.csv")

# Adicionar os dados ao banco
for _, row in df.iterrows():
    employee = Employee(
        first_name=row["FIRST NAME"],
        last_name=row["LAST NAME"],
        sex=SexEnum(row["SEX"]),
        doj=datetime.strptime(row["DOJ"], "%m/%d/%Y"),
        current_date=datetime.strptime(row["CURRENT DATE"], "%m/%d/%Y"),
        designation=DesignationEnum(row["DESIGNATION"].replace(" ", "_").upper()),
        age=row["AGE"],
        salary=row["SALARY"],
        unit=UnitEnum(row["UNIT"].upper()),
        leaves_used=row["LEAVES USED"],
        leaves_remaining=row["LEAVES REMAINING"],
        ratings=row["RATINGS"],
        past_exp=row["PAST EXP"]
    )
    session.add(employee)

# Confirmar as alterações
session.commit()

# Consultar todos os funcionários
employees = session.query(Employee).all()
for emp in employees:
    print(emp)


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
from sqlalchemy import create_engine

# Criar a conexão com o banco de dados SQLite chamado "salarios"
engine = create_engine('sqlite:///salarios.db')

# Testar a conexão
try:
    connection = engine.connect()
    print("Conexão com o banco de dados 'salarios' estabelecida com sucesso!")
    connection.close()
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Enum, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import enum

# Criar a conexão com o banco de dados SQLite chamado "salarios"
engine = create_engine('sqlite:///salarios.db')
Base = declarative_base()

# Definir os Enums
class SexEnum(enum.Enum):
    F = "F"
    M = "M"

class DesignationEnum(enum.Enum):
    ANALYST = "Analyst"
    SENIOR_ANALYST = "Senior Analyst"
    MANAGER = "Manager"

class UnitEnum(enum.Enum):
    IT = "IT"
    FINANCE = "Finance"
    MARKETING = "Marketing"

# Definir a tabela
class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    sex = Column(Enum(SexEnum), nullable=False)
    doj = Column(Date, nullable=False)
    current_date = Column(Date, nullable=False)
    designation = Column(Enum(DesignationEnum), nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnitEnum), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)

# Criar as tabelas no banco de dados
Base.metadata.create_all(engine)

print("Tabelas criadas com sucesso!")


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Caminho local do arquivo CSV (substitua pelo seu caminho)
csv_path = "caminho/para/seu/arquivo/salaries.csv"  # Altere para o caminho correto

# Carregar os dados do CSV
df = pd.read_csv(csv_path)

# Ajustar os valores para se adequarem às colunas ENUM
df['SEX'] = df['SEX'].map({"F": "F", "M": "M"})  # Garantir compatibilidade com o Enum
df['DESIGNATION'] = df['DESIGNATION'].map({
    "Analyst": "Analyst", 
    "Senior Analyst": "Senior Analyst", 
    "Manager": "Manager"
})
df['UNIT'] = df['UNIT'].map({
    "IT": "IT", 
    "Finance": "Finance", 
    "Marketing": "Marketing"
})

# Criar a conexão com o banco de dados SQLite
engine = create_engine('sqlite:///salarios.db')

# Enviar os dados do DataFrame para a tabela employees no banco de dados
df.to_sql('employees', con=engine, if_exists='append', index=False)

# Confirmar que os dados foram inseridos
print("Dados inseridos com sucesso no banco de dados 'salarios.db'!")


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
from sqlalchemy import create_engine

# Criar a conexão com o banco de dados
engine = create_engine('sqlite:///salarios.db')

# Conexão com o banco
with engine.connect() as conn:
    # Query SQL
    query = """
    SELECT DESIGNATION, 
           MIN(SALARY) / 12 AS MIN_SALARY, 
           MAX(SALARY) / 12 AS MAX_SALARY, 
           AVG(SALARY) / 12 AS AVG_SALARY
    FROM employees
    GROUP BY DESIGNATION
    """
    # Executar a query
    result = conn.execute(query)
    
    # Exibir os resultados
    for row in result:
        print(row)


In [ ]:
import pandas as pd

# Query SQL
query = """
SELECT DESIGNATION, 
       MIN(SALARY) / 12 AS MIN_SALARY, 
       MAX(SALARY) / 12 AS MAX_SALARY, 
       AVG(SALARY) / 12 AS AVG_SALARY
FROM employees
GROUP BY DESIGNATION
"""

# Executar a query com pandas
df_sql = pd.read_sql_query(query, engine)

# Exibir os resultados
print(df_sql)


In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
from sqlalchemy import select

# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()

# Construir a query com SQLAlchemy ORM
query = (
    select(
        employees.c.DESIGNATION,
        func.min(employees.c.SALARY) / 12,  # MIN(SALARY) / 12
        func.max(employees.c.SALARY) / 12,  # MAX(SALARY) / 12
        func.avg(employees.c.SALARY) / 12   # AVG(SALARY) / 12
    )
    .group_by(employees.c.DESIGNATION)
)

# Executar a query com ORM
result_orm = session.execute(query).fetchall()

# Exibir os resultados
for row in result_orm:
    print(row)

# Fechar a sessão
session.close()
